### Прочитаем датасет из CSV файла

In [ ]:
%pip install -r requirements.txt

In [ ]:
import pandas as pd
from gensim.models import Word2Vec
import numpy as np

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hetulmehta/website-classification")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Monrois\.cache\kagglehub\datasets\hetulmehta\website-classification\versions\1


In [ ]:
df = pd.read_csv(path + '/website_classification.csv')

In [ ]:
df = df[df["Category"] != "Adult"]
print(df)

      Unnamed: 0                                        website_url  \
0              0     https://www.booking.com/index.html?aid=1743217   
1              1                   https://travelsites.com/expedia/   
2              2               https://travelsites.com/tripadvisor/   
3              3              https://www.momondo.in/?ispredir=true   
4              4  https://www.ebookers.com/?AFFCID=EBOOKERS-UK.n...   
...          ...                                                ...   
1387        1387                       http://www.electroshops.com/   
1388        1388                         http://www.cleanridge.com/   
1389        1389                   http://www.creativepetgifts.com/   
1390        1390                           http://www.htmarket.com/   
1391        1391    https://thrivemarket.com/web/membership/welcome   

                                   cleaned_website_text            Category  
0     official site good hotel accommodation big sav...              

In [ ]:
df['tokens_count'] = df['tokens'].apply(lambda x: len(x))
print(df.groupby('Category')['tokens_count'].mean())

KeyError: 'tokens'

### Обучаем Word2Vec модель

In [ ]:
sentences = [text.split() for text in df['cleaned_website_text']]  # Adjust based on your token format

# Train Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

### Создаем векторы для каждого документа

In [ ]:
def document_vector(model, doc):
    # Remove out-of-vocabulary words
    doc = [word for word in doc if word in model.wv]
    if not doc:
        return np.zeros(model.vector_size)
    return np.mean(model.wv[doc], axis=0)

X = np.array([document_vector(model, doc) for doc in sentences])
y = df['Category'].values  # Your labels

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Assuming you have X (word2vec document vectors) and y (labels) from previous steps

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Convert to PyTorch tensors
X_tensor = torch.FloatTensor(X)
y_tensor = torch.LongTensor(y_encoded)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_tensor, y_tensor, test_size=0.2, random_state=42
)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Define the neural network
class TextClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
input_dim = X.shape[1]
num_classes = len(le.classes_)
model = TextClassifier(input_dim, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Training loop
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
    
    # Validation
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Train Loss: {train_loss/len(train_loader.dataset):.4f}')
    print(f'Test Loss: {test_loss/len(test_loader.dataset):.4f}')
    print(f'Accuracy: {100 * correct/total:.2f}%')

ModuleNotFoundError: No module named 'torch'